In [ ]:
import pandas as pd
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
import matplotlib.pyplot as plt

In [ ]:
confirmed = pd.read_csv('raw_data/COVID/time_series_covid19_confirmed_global.csv')
indicators = pd.read_excel('raw_data/SUSTAIN database_08Jan2021_Asia and Latin America.xlsx')
policy = pd.read_excel('raw_data/SUSTAIN database_09Jan2021_policies_Asia and Latin America.xlsx')

In [ ]:
# confirmed.head(10)
# len(list(set(policy.entity)))
# len(list(set(indicators.Country)))
# len(list(set(confirmed['Country/Region'])))
countries = list(set(policy.entity).intersection(set(confirmed['Country/Region'])).intersection(set(indicators['Country'])))

In [ ]:
def create_dataset(num_series, num_steps):
    # size = (num_series, num_steps)
    
    

In [ ]:
confirmed_series = confirmed.groupby('Country/Region').sum().drop(['Lat', 'Long'],axis=1).loc[countries, :]
target = confirmed_series.values

In [ ]:
target.shape

In [ ]:
# define the parameters of the dataset
custom_ds_metadata = {'num_series': len(countries),
                      'num_steps': len(confirmed_series),
                      'prediction_length': 10,
                      'freq': '1D',
                      'start': [pd.Timestamp(confirmed_series.index[0], freq='1D')
                                for _ in range(len(confirmed_series))]
                     }

In [ ]:
train_ds = ListDataset([{FieldName.TARGET: target,
                         FieldName.START: start,
                         FieldName.FEAT_DYNAMIC_REAL: [fdr],
#                          FieldName.FEAT_STATIC_CAT: [fsc]
                        }
                        for (target, start, fdr) in zip(target[:, :-custom_ds_metadata['prediction_length']],
                                                             custom_ds_metadata['start'],
                                                             target[:, :-custom_ds_metadata['prediction_length']],
#                                                              feat_static_cat
                                                            )],
                      freq=custom_ds_metadata['freq'])

In [ ]:
test_ds = ListDataset([{FieldName.TARGET: target,
                        FieldName.START: start,
                        FieldName.FEAT_DYNAMIC_REAL: [fdr],
#                         FieldName.FEAT_STATIC_CAT: [fsc]
                       }
                       for (target, start, fdr) in zip(target,
                                                            custom_ds_metadata['start'],
                                                            target,
#                                                             feat_static_cat
                                                      )],
                     freq=custom_ds_metadata['freq'])

In [ ]:
train_entry = next(iter(train_ds))
train_entry.keys()

In [ ]:
test_entry = next(iter(test_ds))
test_entry.keys()

In [ ]:
test_series = to_pandas(test_entry)
train_series = to_pandas(train_entry)

fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 7))

train_series.plot(ax=ax[0])
ax[0].grid(which="both")
ax[0].legend(["train series"], loc="upper left")

test_series.plot(ax=ax[1])
ax[1].axvline(train_series.index[-1], color='r') # end of train dataset
ax[1].grid(which="both")
ax[1].legend(["test series", "end of train series"], loc="upper left")

plt.show()

In [1]:
from gluonts.transform import (
    AddAgeFeature,
    AddObservedValuesIndicator,
    Chain,
    ExpectedNumInstanceSampler,
    InstanceSplitter,
    SetFieldIfNotPresent,
)